<a href="https://colab.research.google.com/github/CSID-DGU/2021-1-OSSP1-Debugger-4/blob/autoencoder/CNN_AUTOENCODER_ver%20(Transfer_Learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rsFeNH-ldl3-UCjRoEMUtuxPekRjTEgu?usp=sharing)

# **Mounting Goolge Drive**

In [1]:
from google.colab import drive #구글드라이브 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 라이브러리 설정
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import cv2
import gc
import glob
from tensorflow.keras.layers import Reshape
from keras.models import load_model

# 랜덤 시드 고정
SEED=2021
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)


# **안경 미착용 Dataset**

In [ ]:
import sys, os

num = 0
index = []
for a in range(100):
  for i in range(1): #S001만
    #print('S' + str(i + 1))
    for j in range(4): #L1-L4
      #print('L' + str(j + 1))
      for k in range(2): #E1-E2
        #print('E' + str(k + 1))
        for x in range(7): #C달고있는 12개 중 7개만
          index.append(num)
          num = num + 1
        num = num + 5
    num = num + 336  
print(len(index))
print(index)

rootpath = ['./drive/MyDrive/A_Complete', './drive/MyDrive/B_complete', './drive/MyDrive/C_Complete', './drive/MyDrive/D_Complete']
filename = ['/A_', '/B_', '/A_', '/D_']

filepath = []

for i in range(4):
  temp = rootpath[i] + filename[i]
  for j in range(len(index)):
    path = temp + str(index[j]) + '.jpg'
    filepath.append(path)

print(len(filepath))

In [ ]:
!pip install pyturbojpeg  

from turbojpeg import TurboJPEG
jpeg = TurboJPEG()

train_y = []
train_test = []

for _ in range(0,len(filepath)):
  print(_)
  try:
    in_file = open(filepath[_], 'rb')
    img = jpeg.decode(in_file.read(),0)
    in_file.close()
    train_y.append(img.astype("float32")/255.0)
  except:
    print(filepath[_])


Y_1 = np.array(train_y)

np.savez_compressed(f'./drive/MyDrive/dataSet_Non', Y_1)
print(Y_1.shape)





# **안경 착용 Dataset** 

In [ ]:
num = 0
index = []
for a in range(100):
  for i in range(1): #S002만
    #print('S' + str(i + 1))
    for j in range(4): #L1-L4
      #print('L' + str(j + 1))
      for k in range(2): #E1-E2
        #print('E' + str(k + 1))
        for x in range(7): #C달고있는 12개 중 7개만
          index.append(num)
          num = num + 1
        num = num + 5
    num = num + 336

index = [i+144 for i in index]
print(len(index))
print(index)

rootpath = ['./drive/MyDrive/A_Complete', './drive/MyDrive/B_complete', './drive/MyDrive/C_Complete', './drive/MyDrive/D_Complete']
filename = ['/A_', '/B_', '/A_', '/D_']

filepath = []

for i in range(4):
  temp = rootpath[i] + filename[i]
  for j in range(len(index)):
    path = temp + str(index[j]) + '.jpg'
    filepath.append(path)

print(len(filepath))

In [ ]:
!pip install pyturbojpeg  

from turbojpeg import TurboJPEG
jpeg = TurboJPEG()

train_y = []
train_test = []

for _ in range(0,len(filepath)):
  if _%500 == 0:
    print(_)
  try:
    in_file = open(filepath[_], 'rb')
    img = jpeg.decode(in_file.read(),0)
    in_file.close()
    train_y.append(img.astype("float32")/255.0)
  except:
    print(filepath[_])


Y_1 = np.array(train_y)

np.savez_compressed(f'./drive/MyDrive/dataSet_glasses', Y_1)
print(Y_1.shape)


# **NPZ 데이터 Load 후 Y_1에 저장**

In [ ]:
import random
load_y=np.load('./drive/MyDrive/dataSet_Non.npz')  #이미지 데이터가 저장된 NPZ 로드하기.
Y_1 =load_y['arr_0']
load_y.close()
np.random.shuffle(Y_1)

Y_val = Y_1[-2100:-100]    
Y_test = Y_1[-100:] 
Y_1 = Y_1[:-2100]

del load_y
print(Y_1.shape)
gc.collect()

(19920, 128, 128, 3)


689

# **동영상 프레임 추출 데이터 불러오기 (for 전이학습)**

In [ ]:
#팀장 데이터 불러오기 (Pre-Trained Model에 추가학습용) -> 동영상 프레임 추출 사진
minsu = glob.glob('./drive/MyDrive/videoimage3/*.png')
test_minsu = []
for _ in range(0,len(minsu)):
  img = cv2.imread(minsu[_])
  img = cv2.resize(img,(128,128))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  test_minsu.append(img.astype("float32")/255.0)
  
test_minsu = np.array(test_minsu)
np.random.shuffle(test_minsu)

print(test_minsu.shape)

(1912, 128, 128, 3)


# **실제 INPUT Data Load (for testing)**

In [ ]:
#실제 합성된 사진 테스트용 코드
real_test = glob.glob('./*.png')
real = []
img=cv2.imread(real_test[0])
img = cv2.resize(img,(128,128))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
real.append(img.astype("float32")/255.0)


img=cv2.imread(real_test[1])
img = cv2.resize(img,(128,128))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
real.append(img.astype("float32")/255.0)
  
real = np.array(real)

print(real.shape)

# **동영상 프레임 추출 Data + Input Data (For 전이학습)**

In [ ]:
real = []
img=cv2.imread(real_test[1])
img = cv2.resize(img,(128,128))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
real.append(img.astype("float32")/255.0)
real=real*5
real = np.array(real)

real = np.vstack((real,test_minsu))
print(real.shape)
#np.random.shuffle(real)

(1917, 128, 128, 3)


# **AutoEncoder Using CNN (For train pre-trained model with K-FACE Dataset)**

In [ ]:
# 오토인코더 모델 정의
def Autoencoder():

  X = tf.keras.layers.Input(shape=[128,128,3])

  #Encoder (Convolution layer 4겹)
  H = tf.keras.layers.Conv2D(32,(4,4),(2,2),activation='relu',padding='same')(X)
  H = tf.keras.layers.Conv2D(32*2,(4,4), (2,2),activation='relu',padding='same')(H)
  H = tf.keras.layers.Conv2D(32*4, (4,4),(2,2),activation='relu',padding='same')(H)
  H = tf.keras.layers.Conv2D(32*8, (4,4),(2,2),activation='relu',padding='same')(H)
  H = tf.keras.layers.Conv2D(32*16, (4,4),(2,2),activation='relu',padding='same')(H)

 

  pool_tmp = H.shape.as_list()
  pool_tmp=pool_tmp[1:]
  
  #Fully-Connected
  H = tf.keras.layers.Flatten()(H)
  flatten_tmp=H.shape.as_list()
  flatten_tmp=flatten_tmp[1]
 
  H = tf.keras.layers.Dense(4086,activation='relu')(H)
  H = tf.keras.layers.Dense(flatten_tmp,activation='relu')(H)
  H = Reshape(pool_tmp)(H)


  #Decoder
  
  H = tf.keras.layers.Conv2DTranspose(32*8,(2,2),strides=(2,2),padding='same',activation='relu')(H)
  H = tf.keras.layers.Conv2DTranspose(32*4,(2,2),strides=(2,2),padding='same',activation='relu')(H)
  H = tf.keras.layers.Conv2DTranspose(32*2,(2,2),strides=(2,2),padding='same',activation='relu')(H)
  H = tf.keras.layers.Conv2DTranspose(32,(2,2),strides=(2,2),padding='same',activation='relu')(H)
  H = tf.keras.layers.Conv2DTranspose(3,(2,2),strides=(2,2),padding='same',activation='relu')(H)

  model = tf.keras.models.Model(X,H)
  model.compile(optimizer='adam',loss='mean_squared_logarithmic_error', metrics=['acc']) #MSLE Loss Funciton 적용

  return model

ae_model = Autoencoder()
ae_model.summary()

# **Load the model (Pre-trained Model)**

In [ ]:
ae_model = load_model('./drive/MyDrive/mnist_mlp_model.h5')

# **Transfer Training Encoder Freeze Ver.**

In [ ]:
#전이학습 모델 생성 인코더 동결  VER.
X = tf.keras.layers.Input(shape=[128,128,3]) #input layer 생성
H = ae_model.layers[1](X)                    #input layer 와 pre-trained model의 첫 번째 layer와 연결

for i in range(2,len(ae_model.layers)):      #모든 layer를 이어 붙인다.
  H=ae_model.layers[i](H)

new_model = tf.keras.models.Model(X,H)       #새로운 모델 new_model 정의

for i in range(0,len(ae_model.layers)):      #모든 layer의 trainable을 False 로 설정
  new_model.layers[i].trainable=False

for i in range(10,len(ae_model.layers)):     #Encoder, FC Freeze, But Decoder set trainable
  new_model.layers[i].trainable=True

#model의 learning rate를 매우 낮게 설정해 fine tuning 실시.
new_model.compile(optimizer=keras.optimizers.Adam(1e-4),loss='mean_squared_logarithmic_error', metrics=['acc']) 
new_model.summary()

# **Transfer Training Decoder Freeze Ver.**

In [ ]:
#전이학습 모델 생성 디코더 동결  VER.
X = tf.keras.layers.Input(shape=[128,128,3]) #input layer 생성
H = ae_model.layers[1](X)                    #input layer 와 pre-trained model의 첫 번째 layer와 연결

for i in range(2,len(ae_model.layers)):      #모든 layer를 이어 붙인다.
  H=ae_model.layers[i](H)

new_model = tf.keras.models.Model(X,H)       #새로운 모델 new_model 정의

for i in range(0,len(ae_model.layers)):      #모든 layer의 trainable을 False 로 설정
  new_model.layers[i].trainable=False

for i in range(1,6:     #Encoder, FC Freeze, But Decoder set trainable
  new_model.layers[i].trainable=True

#model의 learning rate를 매우 낮게 설정해 fine tuning 실시.
new_model.compile(optimizer=keras.optimizers.Adam(1e-4),loss='mean_squared_logarithmic_error', metrics=['acc']) 
new_model.summary()

# **Transfer Training FC(Fully-Connected) Freeze Ver.**

In [ ]:
#전이학습 모델 생성 Fully-Connected 동결  VER.
X = tf.keras.layers.Input(shape=[128,128,3]) #input layer 생성
H = ae_model.layers[1](X)                    #input layer 와 pre-trained model의 첫 번째 layer와 연결

for i in range(2,len(ae_model.layers)):      #모든 layer를 이어 붙인다.
  H=ae_model.layers[i](H)

new_model = tf.keras.models.Model(X,H)       #새로운 모델 new_model 정의

for i in range(0,len(ae_model.layers)):      #모든 layer의 trainable을 False 로 설정
  new_model.layers[i].trainable=False

for i in range(6,10):     #Encoder, FC Freeze, But Decoder set trainable
  new_model.layers[i].trainable=True

#model의 learning rate를 매우 낮게 설정해 fine tuning 실시.
new_model.compile(optimizer=keras.optimizers.Adam(1e-4),loss='mean_squared_logarithmic_error', metrics=['acc']) 
new_model.summary()

# **Set Model-Checkpoint using Validation accuracy**

In [ ]:
#모델 체크포인트 설정
checkpoint_path = 'checkpoint.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only = True,
                                                save_best_only = True,
                                                monitor='val_acc',
                                                verbose=1)

# **Training the AutoEncoder using CNN with K-FACE DataSet**

In [ ]:
history = ae_model.fit(test_minsu,
                      test_minsu,
                       batch_size=8, #한번에 학습할 데이터 Batch_size
                       epochs=10,      #에포크 설정
                       verbose=1,      #학습과정 시각화                     
                       callbacks=[checkpoint],  #모델 체크포인트 저장
                       validation_split=0.2)         

# **Transfer Learning using Pre-trained model (K-FACE)**

In [ ]:
history = new_model.fit(real,
                      real,
                       batch_size=8, #한번에 학습할 데이터 Batch_size
                       epochs=1000,      #에포크 설정
                       verbose=1,      #학습과정 시각화                       
                       callbacks=[checkpoint],  #모델 체크포인트 저장
                       validation_split=0.2) 


# **Testing the Pre-trained Model (K-FACE Dataset)**

In [ ]:
ae_images = ae_model.predict(test_minsu) #전이학습 평가
ae_images.shape

(1912, 128, 128, 3)

In [ ]:
ae_images = ae_model.predict(real)  #최종 목표 합성물 평가
ae_images.shape

In [ ]:
ae_images = ae_model.predict(Y_test)  #Pre-trained Model 평가용
ae_images.shape

# **Testing the Transfer Learning Model**

In [ ]:
new_images = new_model.predict(test_minsu) #전이학습 평가
new_images.shape

(1912, 128, 128, 3)

In [ ]:
new_images = new_model.predict(real)  #최종 목표 합성물 평가
new_images.shape

(1917, 128, 128, 3)

In [ ]:
new_images = new_model.predict(Y_test)  #Pre-trained Model 평가용
new_images.shape

(100, 128, 128, 3)

# **Show the Image (Pre-trained Model)**

In [ ]:
#이미지 출력
num = 10 
plt.figure(figsize=(20,8))
for i in range(10):
    # 원본 이미지
    ax = plt.subplot(2, num, i+1)
    plt.imshow(real[i*10])
    plt.title("Original %s" % str(i))
    plt.axis('off')
    
    # 복원 이미지
    ax = plt.subplot(2, num, i+num+1)
    plt.imshow(ae_images[i*10])
    plt.title("Auto-encoded %s" % str(i))
    plt.axis('off')
    
plt.show()

# **Show the Image (Transfre Learning Model)**

In [ ]:
#이미지 출력
num = 10 
plt.figure(figsize=(20,8))
for i in range(10):
    # 원본 이미지
    ax = plt.subplot(2, num, i+1)
    plt.imshow(real[i*10])
    plt.title("Original %s" % str(i))
    plt.axis('off')
    
    # 복원 이미지
    ax = plt.subplot(2, num, i+num+1)
    plt.imshow(ae_images[i*10])
    plt.title("Auto-encoded %s" % str(i))
    plt.axis('off')
    
plt.show()

# **Show the Loss and Validation_Loss**

In [ ]:
plt.figure(figsize=(12,9))
plt.plot(np.arange(1,len(history.history['loss'])+1), history.history['loss'])
plt.plot(np.arange(1,len(history.history['val_loss'])+1), history.history['val_loss'])
plt.title('Loss / Val Loss', fontsize=20, color='red')
plt.xlabel('Epochs', color='red')
plt.ylabel('Loss',color='red')
plt.legend(['Loss', 'val_loss'], fontsize=15)
plt.show()

# **Save the Model (Pre-trained Model)**

In [ ]:
from keras.models import load_model
ae_model.save('./drive/MyDrive/mnist_mlp_model.h5')

# **Fine tuning (Pre-trained Model)**

In [ ]:
import keras
ae_model.compile(optimizer=keras.optimizers.Adam(1e-5),loss='mean_squared_error', metrics=['acc'])

# **Write the images**

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

cv2.imwrite('real_test.jpg', cv2.cvtColor(test_minsu[1]*255, cv2.COLOR_BGR2RGB))
cv2_imshow(cv2.cvtColor(test_minsu[1]*255, cv2.COLOR_BGR2RGB))

